In [1]:
# %load MLP_Skeleton.py
"""
SANAD SAHA
933 620 612
"""
from __future__ import division
from __future__ import print_function

import sys
try:
   import _pickle as pickle
except:
   import pickle

import numpy as np

In [2]:
# This is a class for a LinearTransform layer which takes an input 
# weight matrix W and computes W x as the forward step
class LinearTransform(object):

    def __init__(self, W, b):
    # DEFINE __init function
        self.W = W
        self.b = b
        
    def forward(self, x):
    # DEFINE forward function
        Z = np.dot(x, self.W) + self.b
        return Z
    
        
        
    def backward(
        self, 
        grad_output, 
        learning_rate=0.0, 
        momentum=0.0, 
        l2_penalty=0.0,
    ):
    # DEFINE backward function
        print("Backward not completed")
# ADD other operations in LinearTransform if needed

In [3]:
# This is a class for a ReLU layer max(x,0)
class ReLU(object):

    def forward(self, x):
        #x[x < 0.0] = 0.0
        x = np.maximum(x, 0)
        return x

    def backward(
        self, 
        grad_output, 
        learning_rate=0.0, 
        momentum=0.0, 
        l2_penalty=0.0,
    ):
    # DEFINE backward function
        print("relu backprop not done")
# ADD other operations in ReLU if needed

In [4]:
# This is a class for a sigmoid layer followed by a cross entropy layer, the reason 
# this is put into a single layer is because it has a simple gradient form
class SigmoidCrossEntropy(object):
    def forward(self, x):
        self.x = 1 / (1 + np.exp(-x))
        return self.x
    
    def backward(self, grad_output, learning_rate=0.0, momentum=0.0, l2_penalty=0.0):
        print("Nothing")
        # DEFINE backward function
    
# ADD other operations and data entries in SigmoidCrossEntropy if needed
    def compute_cost(self, y, target_y):
        m = target_y.shape[1]
        logprobs = np.multiply(np.log(y),target_y)/m +  np.multiply(np.log(1 - y), 1- target_y)/m
        cost = - np.sum(logprobs) 
        cost = np.squeeze(cost)
        return cost 

In [5]:
# This is a class for the Multilayer perceptron
class MLP(object):

    def __init__(self, input_dims, hidden_units):
    # INSERT CODE for initializing the network
        self.hidden_units = hidden_units
        self.input_dims = input_dims
        #initialize W1, w2, b1, b2
        
        self.w1 = np.random.randn(self.input_dims, self.hidden_units) * np.sqrt(2/(self.input_dims - 1))
        self.b1 = np.zeros((1, self.hidden_units))
        self.w2 = np.random.randn(self.hidden_units, 1) * np.sqrt(2/(self.hidden_units - 1))
        self.b2 = np.zeros((1, 1))

#         assert (self.w1.shape == (self.hidden_units, self.input_dims))
#         assert (self.b1.shape == (self.hidden_units, 1))
#         assert (self.w2.shape == (1, self.hidden_units))
#         assert (self.b2.shape == (1, 1))
        
    def train(self, x_batch, y_batch, learning_rate = 0.0, momentum = 0.0, l2_penalty = 0.0):
    # INSERT CODE for training the network
        m = x_batch.shape[0]
        
        lt1 = LinearTransform(self.w1, self.b1)
        z1 = lt1.forward(x_batch)
#         sc = SigmoidCrossEntropy()
#         a1 = sc.forward(z1)
        r = ReLU()
#         print(z1[:3, :4])
        a1 = r.forward(z1)
#         print(a1.shape)
#         print(a1[:3, :4])
        #print(x_batch[1:10, 1:10])
        #print(self.w1[3, 1:3])
        
        
        lt2 = LinearTransform(self.w2, self.b2)
        z2 = lt2.forward(a1)
#         print(z2.shape)
#         print(z2[:10, :])
        sigmoid_cross_entropy = SigmoidCrossEntropy()
        a2 = sigmoid_cross_entropy.forward(z2)
#         print(a2.shape)
#         print(a2[:10, :])
        #self.eeval(a2, y_batch)
        
        #loss = sigmoid_cross_entropy.compute_cost(a2, y_batch)
        #print("loss ")
        #print(loss)
        #BACKPROP
        assert(a2.shape == y_batch.shape)
        dz2 = a2 - y_batch
        dw2 = np.dot(a1.T, dz2)
        db2 = np.sum(dz2, axis = 0, keepdims = True)
        dw2 /= m
        db2 /= m
        #print(db2)
        
        assert (self.w2.shape == dw2.shape) 
        assert (self.b2.shape == db2.shape)
        
        grad_relu = (z1 > 0.0)
        
        vera = np.dot(dz2, self.w2.T)
        dz1 = np.multiply(vera, grad_relu)
        dw1 = np.dot(x_batch.T, dz1)
        db1 = np.sum(dz1, axis = 0, keepdims = True)
        dw1 /= m
        db1 /= m
        assert(dw1.shape == self.w1.shape)
        assert (self.b1.shape == db1.shape)
       
        #print(a2.shape)
        #print(self.evaluate(a2, y_batch))
        
        #print("Testing")
        
#         print(self.w1[10, :3])
        
        self.w1 = self.w1 - learning_rate * dw1
        self.b1 = self.b1 - learning_rate * db1
        self.w2 = self.w2 - learning_rate * dw2
        self.b2 = self.b2 - learning_rate * db2
        
        return self.eeval(a2, y_batch)
#         print(self.w1[10, :3])
        #print(dw2)
        #print("Match testing ")
        #print(self.w1[10, 1:5])
            
    def eeval(self, y, ycap):
        t = (y > 0.4999999)
#         print(ycap[:5,:])
#         print(t[:5,:])
        pp = np.count_nonzero(t==ycap)
        return(float(pp)/float(y.shape[0])*100)
#         missmatch = np.sum()
#         print(dhon[:5, :])
#         print(y.shape[0])
#         print(missmatch)
#         print(float(missmatch)/float(y.shape[0])*100)
        
    def evaluate(self, x, y):
        
        lt1 = LinearTransform(self.w1, self.b1)
        z1 = lt1.forward(x)
        r = ReLU()
        a1 = r.forward(z1)
        lt2 = LinearTransform(self.w2, self.b2)
        z2 = lt2.forward(a1)
        sigmoid_cross_entropy = SigmoidCrossEntropy()
        a2 = sigmoid_cross_entropy.forward(z2)
        predictions = (a2 > 0.5)
        accuracy = float((np.dot(predictions.T, y) + np.dot(1-predictions.T, 1-y))/float(y.size)*100)
        return accuracy
    
    # INSERT CODE for testing the network
# ADD other operations and data entries in MLP if needed
        


if __name__ == '__main__':
    if sys.version_info[0] < 3:
        data = pickle.load(open('cifar_2class_py2.p', 'rb'))
    else:
        data = pickle.load(open('cifar_2class_py2.p', 'rb'), encoding='bytes')
    
    train_x = data[b'train_data']
    train_y = data[b'train_labels']
    test_x = data[b'test_data']
    test_y = data[b'test_labels']

    num_examples, input_dims = train_x.shape
    #print(train_x.shape)
    #normalizing the input
    x_norm = np.linalg.norm(train_x, ord = 2, axis = 0, keepdims = True)
    
    # Divide x by its norm.
    train_x = train_x/x_norm
#     train_x = train_x - train_x.mean()
#     train_x = train_x / train_x.var()
#     print(train_x[0, :4])
#     print(train_y[:10,:])
#     print(train_x.shape)
#     print(train_y.shape)
#     print (train_x)
    
    # INSERT YOUR CODE HERE
    # YOU CAN CHANGE num_epochs AND num_batches TO YOUR DESIRED VALUES
    num_epochs = 10
    num_batches = 1000
    hidden_units = 50
    
    mlp = MLP(input_dims, hidden_units)
    for i in range (6000):
        
        b = mlp.train(train_x[:1000, :], train_y[:1000, :], 0.3)
        
        if(i % 300 == 0):
            print(b)
    
  
    
      
#     frm = epoch * num_batches
#     to = (epoch+1) * num_batches 
#     mlp.train(train_x[frm:to, :].T, train_y[frm:to, :].T, 1.5)
    
#     for epoch in range(num_epochs):
  
#     #INSERT YOUR CODE FOR EACH EPOCH HERE
        
    
#         #train_accuracy = evaluate()
#         for b in range(num_batches):
#             total_loss = 0.0
#             # INSERT YOUR CODE FOR EACH MINI_BATCH HERE
#             mlp.train(train_x[b * 10: (b+1) * 10, :], train_y[b * 10: (b+1) * 10, :], 0.005)
#             # MAKE SURE TO UPDATE total_loss
# #             print(
# #                 '\r[Epoch {}, mb {}]    Avg.Loss = {:.3f}'.format(
# #                     epoch + 1,
# #                     b + 1,
# #                     total_loss,
# #                 ),
# #                 end='',
# #             )
# #             sys.stdout.flush()
#         train_accuracy = evaluate(train_x, train_y)
        # INSERT YOUR CODE AFTER ALL MINI_BATCHES HERE
        # MAKE SURE TO COMPUTE train_loss, train_accuracy, test_loss, test_accuracy
#         print()
#         print('    Train Loss: {:.3f}    Train Acc.: {:.2f}%'.format(
#             train_loss,
#             100. * train_accuracy,
#         ))
#         print('    Test Loss:  {:.3f}    Test Acc.:  {:.2f}%'.format(
#             test_loss,
#             100. * test_accuracy,
#         ))


50.6
63.9
64.2
66.3
68.2
69.3
70.7
69.1
70.1
70.6
71.39999999999999
71.89999999999999
72.39999999999999
73.2
73.6
75.0
74.4
75.1
75.9
75.7


In [6]:
class test(object):
    def __init__(self, x):
        self.x = x
        
    def change(self, p):
        self.x = p  
        
    def show(self):
        print(self.x)
        
    def add(self, y):
        self.x = self.x + y;
        return self.x;
        


In [7]:
# t = test(5)
# t.show()
# t.change(20)
# t.show()
# t2 = test(100)
# t2.show()
# t2.change(-10)
# t2.show()
# t.change(10)
# print(t2.add(50))
# print(t.add(20))
# t.show()
pp = np.random.randn(5,5) * 0.01
print(pp)
pp = np.maximum(pp, 0)
print(pp)

[[ 0.00019625  0.01292163 -0.00461242  0.01164016 -0.00482791]
 [-0.0033735   0.02425993  0.0082251  -0.00367694  0.0042211 ]
 [ 0.00021651 -0.02202308  0.00196516 -0.00436201 -0.01342334]
 [-0.00373149  0.0047105  -0.00043725 -0.00994808  0.01175125]
 [ 0.00875227 -0.00092618 -0.00659876 -0.01794115  0.00930897]]
[[0.00019625 0.01292163 0.         0.01164016 0.        ]
 [0.         0.02425993 0.0082251  0.         0.0042211 ]
 [0.00021651 0.         0.00196516 0.         0.        ]
 [0.         0.0047105  0.         0.         0.01175125]
 [0.00875227 0.         0.         0.         0.00930897]]
